![Python](https://img.shields.io/badge/python-3.9-blue)
![Status: Pending Migration](https://img.shields.io/badge/status-pending%20migration-orange)

<a id="table-of-contents"></a>
# 📖 Table of Contents

[🧭 Objective](#objective)  
- [🧠 What is Clustering?](#what-is-clustering)  
- [📌 When is Clustering Useful?](#clustering-use-cases)  
- [📏 Evaluation Challenges](#evaluation-challenges)

[📦 Data Setup](#data-setup)  
- [📥 Load Dataset](#load-dataset)  
- [🧹 Preprocessing](#preprocessing)

[📊 Clustering Algorithms](#clustering-algos)

[📈 KMeans](#kmeans)  
- [⚙️ Core Logic](#kmeans-logic)  
- [📉 Elbow + Silhouette Method](#kmeans-eval)  
- [📊 Visual Output](#kmeans-visuals)

[🧱 Hierarchical Clustering](#hierarchical)  
- [🌳 Dendrograms](#dendrograms)  
- [📏 Linkage Types](#linkage)  
- [📊 Cluster Cuts](#cluster-cuts)

[🌐 DBSCAN](#dbscan)  
- [🔍 Intuition](#dbscan-intuition)  
- [⚙️ Epsilon, MinPts](#dbscan-params)  
- [📊 Visual Output](#dbscan-visuals)

[🎲 Gaussian Mixture Models (GMM)](#gmm)  
- [🧠 Soft Clustering](#soft-clustering)  
- [📈 BIC / AIC for Model Selection](#gmm-eval)

[📍 Mean Shift](#meanshift)  
- [📏 Bandwidth Selection](#bandwidth)

[🎼 Spectral Clustering](#spectral)  
- [🔗 Graph Representation](#spectral-graph)  
- [📊 Laplacian & Eigenvectors](#spectral-laplacian)

[📡 OPTICS](#optics)  
- [🪜 Reachability Plot](#reachability)

[🌲 BIRCH](#birch)

[🔥 HDBSCAN](#hdbscan)

[🧬 Other Methods](#other-methods)  
- [🧿 Affinity Propagation](#affinity-prop)  
- [🧊 Ward’s Method](#ward)

[📌 Summary Table](#summary-table)  
- [📋 Comparison Across Methods](#method-comparison)  
- [🧭 Practical Recommendations](#recommendations)

[❓ FAQ / Notes](#faq)  
- [📏 Choosing Number of Clusters](#choose-k)  
- [📊 Distance Metrics](#distance-metrics)  
- [⚠️ Common Pitfalls](#pitfalls)

___

<a id="objective"></a>
# 🧭 Objective

<a id="what-is-clustering"></a>
#### 🧠 What is Clustering?

<a id="clustering-use-cases"></a>
#### 📌 When is Clustering Useful?

<a id="evaluation-challenges"></a>
#### 📏 Evaluation Challenges

[Back to the top](#table-of-contents)
___



<a id="data-setup"></a>
# 📦 Data Setup

<a id="load-dataset"></a>
#### 📥 Load Dataset

<a id="preprocessing"></a>
#### 🧹 Preprocessing

[Back to the top](#table-of-contents)
___



<a id="clustering-algos"></a>
# 📊 Clustering Algorithms

[Back to the top](#table-of-contents)
___



<a id="kmeans"></a>
# 📈 KMeans

<a id="kmeans-logic"></a>
#### ⚙️ Core Logic

<a id="kmeans-eval"></a>
#### 📉 Elbow + Silhouette Method

<a id="kmeans-visuals"></a>
#### 📊 Visual Output

[Back to the top](#table-of-contents)
___



<a id="hierarchical"></a>
# 🧱 Hierarchical Clustering

<a id="dendrograms"></a>
#### 🌳 Dendrograms

<a id="linkage"></a>
#### 📏 Linkage Types

<a id="cluster-cuts"></a>
#### 📊 Cluster Cuts

[Back to the top](#table-of-contents)
___



<a id="dbscan"></a>
# 🌐 DBSCAN

<a id="dbscan-intuition"></a>
#### 🔍 Intuition

<a id="dbscan-params"></a>
#### ⚙️ Epsilon, MinPts

<a id="dbscan-visuals"></a>
#### 📊 Visual Output

[Back to the top](#table-of-contents)
___



<a id="gmm"></a>
# 🎲 Gaussian Mixture Models (GMM)

<a id="soft-clustering"></a>
#### 🧠 Soft Clustering

<a id="gmm-eval"></a>
#### 📈 BIC / AIC for Model Selection

[Back to the top](#table-of-contents)
___



<a id="meanshift"></a>
# 📍 Mean Shift

<a id="bandwidth"></a>
#### 📏 Bandwidth Selection

[Back to the top](#table-of-contents)
___



<a id="spectral"></a>
# 🎼 Spectral Clustering

<a id="spectral-graph"></a>
#### 🔗 Graph Representation

<a id="spectral-laplacian"></a>
#### 📊 Laplacian & Eigenvectors

[Back to the top](#table-of-contents)
___



<a id="optics"></a>
# 📡 OPTICS

<a id="reachability"></a>
#### 🪜 Reachability Plot

[Back to the top](#table-of-contents)
___



<a id="birch"></a>
# 🌲 BIRCH

[Back to the top](#table-of-contents)
___



<a id="hdbscan"></a>
# 🔥 HDBSCAN

[Back to the top](#table-of-contents)
___



<a id="other-methods"></a>
# 🧬 Other Methods

<a id="affinity-prop"></a>
#### 🧿 Affinity Propagation

<a id="ward"></a>
#### 🧊 Ward’s Method

[Back to the top](#table-of-contents)
___



<a id="summary-table"></a>
# 📌 Summary Table

<a id="method-comparison"></a>
#### 📋 Comparison Across Methods

<a id="recommendations"></a>
#### 🧭 Practical Recommendations

[Back to the top](#table-of-contents)
___



<a id="faq"></a>
# ❓ FAQ / Notes

<a id="choose-k"></a>
#### 📏 Choosing Number of Clusters

<a id="distance-metrics"></a>
#### 📊 Distance Metrics

<a id="pitfalls"></a>
#### ⚠️ Common Pitfalls

[Back to the top](#table-of-contents)
___

